In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import ipywidgets as widgets
import glob
import os
from IPython.display import clear_output
from IPython.display import display

# If you want to widen the page
# you can modify *width* to the one you prefer

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

def download_sheets(_):
    try:
        gauth = GoogleAuth()
        gauth.LocalWebserverAuth()

        drive = GoogleDrive(gauth)

        file_list = drive.ListFile({'q': "mimeType='application/vnd.google-apps.spreadsheet' and sharedWithMe and 'cboullay@gmail.com' in writers and (title contains 'TP-NOTE' or title contains 'CAML' or title contains 'C#')"}).GetList()
        for file1 in file_list:
            file1.GetContentFile('sheets/' + file1['title'] + '.xlsx', mimetype='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
            print('Downloaded {}'.format(file1['title']))

        print('Done')
    except:
        print("You're missing the client_secrets.json file, go check the README")
    
def on_button_click(_):
    print('Hello')
        
button = widgets.Button(
    description='Download sheets',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='download all correction sheets from your google drive',
)

button.on_click(download_sheets)

display(button)

In [ ]:
# Get files
files = [f for f in glob.glob(os.getcwd() + "/sheets/*.xlsx")]
raw_files = [f.split('/')[-1].split('\\')[-1] for f in files]

# Because I can.
practical_nb = widgets.Dropdown(
    options=raw_files,
    value=raw_files[0],
    rows=10,
    description='practical:',
    disabled=False
)

def read_practical(file):
    # Practical selection

    TP = pd.ExcelFile(file)

    # Grade Processing

    df = dict()
    for sheet in TP.sheet_names[1:]:
        df[sheet] = TP.parse(sheet, header=None)

    # Getting all the different exercises
    cols = []

    for i in range(len(df['promo'].loc[0])):
            exercise = str(df['promo'].loc[int(str(df['promo'].loc[0][i]) == 'nan')][i]) # append second line if first line is nan
            while (exercise in cols and str(exercise) != 'nan'):
                exercise += "_"
            cols.append(exercise)
            
    # Grouping columns two by two, so that each column represent one exercise
    for i in range(len(cols)):
        if str(cols[i]) == 'nan':
            cols[i] = cols[i-1] + "_to_drop"
            
    cols[0] = 'login' # Just in case someone forgets...

    promo = pd.DataFrame()

    # Data cleaning for every sheet in the .xlsx
    for sheet in TP.sheet_names[2 - ('CTRL' in file or 'PART' in file):]:
        df[sheet].columns = cols
        df[sheet].drop([0, 1, 2], inplace=True)
        df[sheet][cols[4:-2]] = df[sheet][cols[4:-2]].apply(lambda a: pd.to_numeric(a, errors='coerce'))
        df[sheet].set_index('login', inplace=True)
        df[sheet]['corrector'].fillna(method='ffill', inplace=True)
        df[sheet].fillna(0, inplace=True)
        for i in range(4, len(cols) - 3, 2):
            df[sheet][cols[i]] += df[sheet][cols[i+1]]
            df[sheet][cols[i]].apply(lambda a: a if a <= 2 else 2) # Check typo in grades
        df[sheet].drop(cols[5:-2:2], axis=1, inplace = True)
        df[sheet]['total'] = df[sheet][cols[4:-2:2]].sum(axis=1) / (len(cols) - 6) * 100
        promo = pd.concat([promo, df[sheet]])
    return promo

practicals = [read_practical(f) for f in files]

In [ ]:
display(practical_nb)

In [ ]:
# Normalization

print(files[raw_files.index(practical_nb.value)])

stats_promo = practicals[raw_files.index(practical_nb.value)].groupby(['gr']).mean()
stats_promo.loc[:, stats_promo.columns != 'total'] *= 50 # Everything is in % now

In [ ]:
# Overview of exercise succes for each class

plt.figure(figsize=(len(stats_promo.columns),8))
sns.heatmap(stats_promo, vmin=0, vmax=100, annot=True, cmap='Greens')

In [ ]:
plt.figure(figsize=(6,4))
sns.kdeplot(stats_promo['total'])

In [ ]:
p = practicals[raw_files.index(practical_nb.value)]

p[p.columns[4: -3]].plot(kind='density', figsize=(8, (len(p.columns) / 1.5)), subplots=True, xlim=(0, 2), ylim=(0, 1))
plt.show()

In [ ]:
group_choice = widgets.Dropdown(
    options=practicals[0]['gr'].unique(),
    value=practicals[0]['gr'].unique()[0],
    rows=10,
    description='group:',
    disabled=False
)

student_choice = widgets.Dropdown(
    options=practicals[0][practicals[0]['gr'] == group_choice.value].index.tolist() + ['nope'],
    value=practicals[0][practicals[0]['gr'] == group_choice.value].index[0],
    rows=10,
    description='student:',
    disabled=False
)

practical_choice = widgets.SelectMultiple(
    options=raw_files,
    value=raw_files,
    rows=10,
    description='practicals:',
    disabled=False
)

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        student_choice.value = 'nope'
        student_choice.options = practicals[0][practicals[0]['gr'] == group_choice.value].index.tolist() + ['nope']
        student_choice.value = student_choice.options[0]
        clear_output()
        display(group_choice)
        display(student_choice)
        
def on_student_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        print("changed to %s" % change['new'])

group_choice.observe(on_change)
student_choice.observe(on_student_change)



In [ ]:
display(group_choice)
display(student_choice)

In [ ]:
student_grades = pd.DataFrame([practicals[i].loc[student_choice.value, ['total']].append(pd.Series([raw_files[i].split('.')[0]], ['TP'])) for i in range(len(raw_files))])
student_grades

In [ ]:
plt.plot(student_grades['total'])

In [ ]:
from bokeh.plotting import output_notebook

output_notebook()

In [ ]:
from bokeh.io import show
from bokeh.models.widgets import Select
from bokeh.layouts import row
from bokeh.models.callbacks import CustomJS

select_sheet = Select(title='Sheets', options=raw_files)
select_group = Select(title='Group', options=practicals[0]['gr'].unique().tolist())
select_student = Select(title='Student', options=practicals[0][practicals[0]['gr'] == group_choice.value].index.tolist())

def change(attr, old, new):
    print('hello')

select_group.on_change('value', change)

show(row(select_group, select_student, select_sheet))

In [ ]:
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure

student_grades['color'] = student_grades.total.apply (lambda grade: 'blue' if grade > 95 else 'red')

source = ColumnDataSource(student_grades)
averages = [practicals[raw_files.index(i)].mean()['total'] for i in raw_files]

p = figure(y_range=student_grades['TP'])
p.hbar(y='TP', source=source, left=0, right='total', color='color', height=0.5)

show(p)

In [ ]:
from bokeh.models.glyphs import HBar

def group_change(attr, old, new):
    s.select_student.options = practicals[0][practicals[0]['gr'] == new].index.tolist()
    s.select_group.value = new
    
def student_change(attr, old, new):
    s.select_student.value = new
    
def sheet_change(attr, old, new):
    s.select_sheet.value = new

class selectors():
    source = ColumnDataSource(student_grades)

    select_sheet = Select(title='Sheets', options=raw_files, value=raw_files[0])
    select_group = Select(title='Group', options=practicals[0]['gr'].unique().tolist())
    select_group.value = select_group.options[0]
    select_student = Select(title='Student', options=practicals[0][practicals[0]['gr'] == select_group.value].index.tolist())
    select_student.value = select_student.options[0]
    
    select_group.on_change('value', group_change)
    select_sheet.on_change('value', sheet_change)
    select_student.on_change('value', student_change)
    
s = selectors()
    
def plaf(doc):
    doc.add_root(row(s.select_group, s.select_student, s.select_sheet))

In [ ]:
show(plaf, notebook_url='http://localhost:8889')

In [ ]:
from bokeh.models.glyphs import HBar

student_grades = pd.DataFrame([practicals[i].loc[s.select_student.value, ['total']].append(pd.Series([raw_files[i].split('.')[0]], ['TP'])) for i in range(len(raw_files))])

student_grades['color'] = student_grades.total.apply (lambda grade: 'blue' if grade > 95 else 'red')

source = ColumnDataSource(student_grades)
averages = [practicals[raw_files.index(i)].mean()['total'] for i in raw_files]

class bar():
    def __init__(self):
        self.p = figure(y_range=student_grades['TP'])
        self.glyph = HBar(y="TP", right="total", left=0, height=0.5, fill_color="#b3de69")
        self.p.add_glyph(source, self.glyph)
    
    def display(self):
        show(self.p)

In [ ]:
b = bar()

b.display()

In [ ]:
from bokeh.models import LinearColorMapper, BasicTicker, PrintfTickFormatter, ColorBar

data = {'exo' : [], 'group' : [], 'total' : []}

for index, cols in stats_promo.iterrows():
    for practical in stats_promo.columns:
        data['exo'].append(practical)
        data['group'].append(index)
        data['total'].append(stats_promo[practical][index])
        
tp_stats = pd.DataFrame(data=data)
practical = raw_files[0]
practical = practical.split('.')[0]

colors = ["#ea9999", "#f4c7c3", "#fce8b2", "#b7e1cd", "#57bb8a"]
mapper = LinearColorMapper(palette=colors, low=tp_stats.total.min(), high=tp_stats.total.max())

p = figure(title="Class averages for practical: {0}".format(practical),
           x_range=stats_promo.columns.tolist(), y_range=[index for index, cols in stats_promo.iterrows()][::-1],
           x_axis_location="above", plot_width=1200, plot_height=600,
           tools="", toolbar_location='below',
           tooltips=[('group', '@group'), ('mean', '@total')])

p.axis.axis_line_color = None
p.axis.major_tick_line_color = None
p.axis.major_label_text_font_size = "10pt"
p.axis.major_label_standoff = 0

p.rect(x="exo", y="group", width=1, height=1,
       source=tp_stats,
       fill_color={'field': 'total', 'transform': mapper},
       line_color='white')

color_bar = ColorBar(color_mapper=mapper, major_label_text_font_size="8pt",
                     ticker=BasicTicker(desired_num_ticks=len(colors)),
                     formatter=PrintfTickFormatter(format="%d%%"),
                     label_standoff=8, border_line_color=None, location=(0, 0))
p.add_layout(color_bar, 'right')

show(p)      # show the plot